In [75]:
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm = tqdm.pandas()

In [78]:
def patient_class(pat_class, payer_orig):
    if pd.isna(pat_class):
        if 'Outpatient' in payer_orig or 'OP' in payer_orig:
            return 'outpatient'
        elif 'Inpatient' in payer_orig or 'IP' in payer_orig:
            return 'inpatient'
        else:
            return pd.NA
    else:
        return pat_class

def payer_cat(x):
    if 'Gross Charges' in x:
        return 'gross'
    elif 'Max Reimbursement' in x:
        return'max'
    elif 'Minimal Reimbursement' in x:
        return'min'
    elif 'Cash Price' in x:
        return 'cash'
    else:
        return 'payer'
    

In [84]:
file = 'EIN_30-1163729_2022_UTHealthNorthHospital_standardcharges.csv'
df = pd.read_csv(file, delimiter='|', skiprows=1, encoding='ansi')

C:\Users\adria\AppData\Local\Temp\ipykernel_3364\4268684706.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, delimiter='|', skiprows=1, encoding='ansi')


In [22]:
df.rename(columns={
    'Procedure': 'internal_code',
    'Code Type': 'code_type',
    'Revenue Code': 'rev_desc',
    'Procedure Description': 'desc',
    'Payer': 'payer_orig',
    'Plan': 'plan_name',
    'Quantity': 'ndc_quantity_desc'
}, inplace=True)

In [38]:
df['code_orig'] = df['internal_code'].where(df['internal_code'].str.startswith('MS'), other=pd.NA)
df['code_prefix'] = np.where(df['code_type'] == 'DRG', 'ms-drg', pd.NA)
df['code'] = df['code_orig'].str.replace('MS', '')
df['code'].fillna('none', inplace=True)
df['code_prefix'].fillna('none', inplace=True)

In [40]:
df['rev_code'] = df['rev_desc'].str.split(" - ").str[0]

In [45]:
cols = df.columns.tolist()
cols = cols[-4:] + cols[:-4]
df = df[cols]

In [59]:
id_vars = cols[:11]

# Define columns to be melted
value_vars = ['Inpatient Expected Reimbursement', 'Outpatient Expected Reimbursement']

# Melt the dataframe 
# Var name as patient_class because the var names don't matter here as these are the rates for `payer_orig`
df_melted = pd.melt(df, id_vars=id_vars, value_vars=value_vars,
                    var_name='patient_class', value_name='rate')

# Extract the patient class from the variable column
df_melted['patient_class'] = df_melted['patient_class'].str.split().str[0].str.lower()

In [67]:
id_vars = cols[:11]
id_vars.remove('payer_orig')
id_vars.remove('plan_name')
# Define columns to be melted
value_vars = cols[11:]
value_vars.remove('Outpatient Expected Reimbursement')
value_vars.remove('Inpatient Expected Reimbursement')

# Melt the dataframe 
# Var name as patient_class because the var names don't matter here as these are the rates for `payer_orig`
df_1 = df.drop(['payer_orig', 'plan_name'], axis=1)
df_melted_2 = pd.melt(df_1, id_vars=id_vars, value_vars=value_vars,
                    var_name='payer_orig', value_name='rate')

# Extract the patient class from the variable column
# df_melted['patient_class'] = df_melted['patient_class'].str.split().str[0].str.lower()

In [71]:
df = pd.concat([df_melted, df_melted_2])

In [76]:
df['patient_class'] = df[['patient_class', 'payer_orig']].progress_apply(lambda x: patient_class(pat_class=x[0], payer_orig=x[1]), axis=1)

100%|██████████| 1545600/1545600 [00:21<00:00, 70457.85it/s]


In [79]:
df['payer_category'] = df['payer_orig'].apply(payer_cat)

In [83]:
df['internal_code'].fillna('na', inplace=True)
df['payer_name'] = df['payer_orig']

,code_orig,code_prefix,code,rev_code,internal_code,code_type,rev_desc,desc,Quantity,payer_orig,plan_name,patient_class,rate,payer_category
0,<NA>,<NA>,<NA>,0637,1000,ERX,0637 - PHARMACY-EXTENSION OF 025X-SELF-ADMINIS...,BENZTROPINE 2 MG TABLET,1 tablet,AARP [1000],AARP [100000],inpatient,NaN,payer
1,<NA>,<NA>,<NA>,0637,1000,ERX,0637 - PHARMACY-EXTENSION OF 025X-SELF-ADMINIS...,BENZTROPINE 2 MG TABLET,1 tablet,AETNA MEDICARE REPLACEMENT [1020],AETNA MEDICARE REPLACEMENT [102002],inpatient,NaN,payer
2,<NA>,<NA>,<NA>,0637,1000,ERX,0637 - PHARMACY-EXTENSION OF 025X-SELF-ADMINIS...,BENZTROPINE 2 MG TABLET,1 tablet,AETNA [1015],AETNA [101529],inpatient,NaN,payer
3,<NA>,<NA>,<NA>,0637,1000,ERX,0637 - PHARMACY-EXTENSION OF 025X-SELF-ADMINIS...,BENZTROPINE 2 MG TABLET,1 tablet,AMERIGROUP MEDICAID [1095],AMERIGROUP MEDICAID [109500],inpatient,NaN,payer
4,<NA>,<NA>,<NA>,0637,1000,ERX,0637 - PHARMACY-EXTENSION OF 025X-SELF-ADMINIS...,BENZTROPINE 2 MG TABLET,1 tablet,AMERIGROUP MEDICARE [1100],AMERIGROUP MEDICARE [110000],inpatient,NaN,payer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236475,MS999,ms-drg,999,NaN,MS999,DRG,NaN,Ungroupable,1,Discounted OP Cash Price,NaN,outpatient,NaN,cash
1236476,MS999,ms-drg,999,NaN,MS999,DRG,NaN,Ungroupable,1,Discounted OP Cash Price,NaN,outpatient,NaN,cash
1236477,MS999,ms-drg,999,NaN,MS999,DRG,NaN,Ungroupable,1,Discounted OP Cash Price,NaN,outpatient,NaN,cash
1236478,MS999,ms-drg,999,NaN,MS999,DRG,NaN,Ungroupable,1,Discounted OP Cash Price,NaN,outpatient,NaN,cash


In [ ]:
df['hospital_ein'] = '30-1163729'
df['hospital_ccn'] = '450690'
df['filename'] = file
df['url'] = 'https://coc.ardenthealthservices.com/2023/easttexas/EIN_30-1163729_2022_UTHealthNorthHospital_standardcharges.csv'
df['file_last_updated'] = '2023-01-10'

In [ ]:
df.to_csv('tyler.csv', index=False)